# $\alpha$=0.1

In [2]:
import torch
import numpy as np
import torchvision.transforms as transforms               
from torchvision.datasets import CIFAR10       
from torch.utils.data import DataLoader
import detectors
import timm
from src.temperature_scaling import ModelWithTemperature
from src.aps import split_data_set, aps_scores, aps_classification, eval_aps

# check GPU status
print("Is CUDA available:", torch.cuda.is_available())
print("Device count:", torch.cuda.device_count())
print("Device name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# load fine-tuned model
model = timm.create_model("resnet50_cifar10", pretrained=True)
model.to(device)

# reprocess the images from CIFAR
data_transform = transforms.Compose([
    transforms.ToTensor(),          # transfer to tensor
    transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2023, 0.1994, 0.2010])  # normalize
])
# load images from CIFAR10
dataset = CIFAR10(root="../../data", train=False, download=True, transform=data_transform)

# Temperature Scaling
temp_scal_loader = DataLoader(dataset, batch_size=32, shuffle=True)
model = ModelWithTemperature(model, temperature=5.0).to(device)
model.set_temperature(temp_scal_loader)

model.eval()

# The number of times the experiment is going to be repeated
num_runs = 10

# error rate
alpha = 0.1

# construct and evaluate repeatedly
all_avg_set_sizes = []
all_avg_coverages = []
print("APS Classification, Start!\n")
for i in range(num_runs):
    print(f"Running experiment {i+1}/{num_runs}...")

    # split dataset
    calib_dataset, test_dataset = split_data_set(dataset, random_seed=i)

    # load data set respectively
    calib_loader = DataLoader(calib_dataset, batch_size=32, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    # calculate q_hat
    calib_scores, _ = aps_scores(model, calib_loader, alpha, device)
    q_hat = np.quantile(calib_scores, 1 - 0.1)  # calculate 1-alpha quantile
    print(f"q_hat = {q_hat}")

    # construct APS
    aps, aps_labels, true_labels = aps_classification(model, test_loader, q_hat, device)

    # evaluate APS
    avg_set_size, avg_coverage = eval_aps(aps_labels, true_labels)
    print(f"Average Prediction Set Size After APS in runs {i+1}: {avg_set_size}")
    print(f"Average Coverage Rate in runs {i+1}: {avg_coverage}\n")

    # record current result
    all_avg_set_sizes.append(avg_set_size)
    all_avg_coverages.append(avg_coverage)

# calculate the final average result
final_avg_set_size = np.mean(all_avg_set_sizes)
final_avg_coverage = np.mean(all_avg_coverages)
final_set_size_std = np.std(all_avg_set_sizes, ddof=0)
final_coverage_std = np.std(all_avg_coverages, ddof=0)

print(f"Final Average Prediction Set Size: {final_avg_set_size:.2f} ± {final_set_size_std:.2f}")
print(f"Final Average Coverage: {final_avg_coverage:.4f} ± {final_coverage_std:.4f}")

Is CUDA available: True
Device count: 1
Device name: NVIDIA GeForce RTX 3060 Ti
Files already downloaded and verified
Before temperature - NLL: 0.354, ECE: 0.046
Optimal temperature: 4.908
After temperature - NLL: 0.827, ECE: 0.420
APS Classification, Start!

Running experiment 1/10...
q_hat = 0.5293287932872773
Total set size: 7097
Total coverage sets: 4496
Total samples amount: 5000
Average Prediction Set Size After APS in runs 1: 1.4194
Average Coverage Rate in runs 1: 0.8992

Running experiment 2/10...
q_hat = 0.5250233471393586
Total set size: 7034
Total coverage sets: 4471
Total samples amount: 5000
Average Prediction Set Size After APS in runs 2: 1.4068
Average Coverage Rate in runs 2: 0.8942

Running experiment 3/10...
q_hat = 0.5251519322395325
Total set size: 7018
Total coverage sets: 4489
Total samples amount: 5000
Average Prediction Set Size After APS in runs 3: 1.4036
Average Coverage Rate in runs 3: 0.8978

Running experiment 4/10...
q_hat = 0.5248161911964416
Total set s

# $\alpha$=0.05

In [3]:
# error rate
alpha = 0.05

# construct and evaluate repeatedly
all_avg_set_sizes = []
all_avg_coverages = []
print("APS Classification, Start!\n")
for i in range(num_runs):
    print(f"Running experiment {i+1}/{num_runs}...")

    # split dataset
    calib_dataset, test_dataset = split_data_set(dataset, random_seed=i)

    # load data set respectively
    calib_loader = DataLoader(calib_dataset, batch_size=32, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    # calculate q_hat
    calib_scores, _ = aps_scores(model, calib_loader, alpha, device)
    q_hat = np.quantile(calib_scores, 1 - alpha)  # calculate 1-alpha quantile
    print(f"q_hat = {q_hat}")

    # construct APS
    aps, aps_labels, true_labels = aps_classification(model, test_loader, q_hat, device)

    # evaluate APS
    avg_set_size, avg_coverage = eval_aps(aps_labels, true_labels)
    print(f"Average Prediction Set Size After APS in runs {i+1}: {avg_set_size}")
    print(f"Average Coverage Rate in runs {i+1}: {avg_coverage}\n")

    # record current result
    all_avg_set_sizes.append(avg_set_size)
    all_avg_coverages.append(avg_coverage)

# calculate the final average result
final_avg_set_size = np.mean(all_avg_set_sizes)
final_avg_coverage = np.mean(all_avg_coverages)
final_set_size_std = np.std(all_avg_set_sizes, ddof=0)
final_coverage_std = np.std(all_avg_coverages, ddof=0)

print(f"Final Average Prediction Set Size: {final_avg_set_size:.2f} ± {final_set_size_std:.2f}")
print(f"Final Average Coverage: {final_avg_coverage:.4f} ± {final_coverage_std:.4f}")

Is CUDA available: True
Device count: 1
Device name: NVIDIA GeForce RTX 3060 Ti
Files already downloaded and verified
Before temperature - NLL: 0.360, ECE: 0.048
Optimal temperature: 4.909
After temperature - NLL: 0.832, ECE: 0.421
APS Classification, Start!

Running experiment 1/10...
q_hat = 0.5950762182474136
Total set size: 9014
Total coverage sets: 4767
Total samples amount: 5000
Average Prediction Set Size After APS in runs 1: 1.8028
Average Coverage Rate in runs 1: 0.9534

Running experiment 2/10...
q_hat = 0.5832848578691483
Total set size: 8622
Total coverage sets: 4718
Total samples amount: 5000
Average Prediction Set Size After APS in runs 2: 1.7244
Average Coverage Rate in runs 2: 0.9436

Running experiment 3/10...
q_hat = 0.5811397880315781
Total set size: 8581
Total coverage sets: 4707
Total samples amount: 5000
Average Prediction Set Size After APS in runs 3: 1.7162
Average Coverage Rate in runs 3: 0.9414

Running experiment 4/10...
q_hat = 0.5867420971393585
Total set s

# Result
$\alpha$=0.1
- Final Average **Prediction Set Size： 1.41**
- Final Average **Coverage: 89.61%**  

$\alpha$=0.05
- Final Average **Prediction Set Size： 1.76**
- Final Average **Coverage: 94.69%**